In [1]:
from langchain_community.llms import Ollama
from datetime import datetime, timezone

llm = Ollama(
    model="phi4:latest",
    base_url="http://localhost:11434",
    temperature=0
)


def call_phi4_single_comment(
    comment_id: str,
    comment_text: str,
    sentiment_group: str,
    created_at: str,
    retries: int,
) -> str:
    prompt = PHI4_SINGLE_COMMENT_PROMPT.format(
        comment_id=comment_id,
        created_at=created_at,
        sentiment_group=sentiment_group,
        comment_text=comment_text.strip()
    )
    for i in range(retries + 1):
        raw = llm.invoke(prompt)

        if raw and raw.strip():
            return raw

        print(f"⚠️ Empty output, retry {i+1}")

    raise RuntimeError("Phi4 returned empty output after retries")


    


/home/mahdi/Sentiment_Analysis_Dima/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_77009/162997373.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(


In [ ]:
PHI4_SINGLE_COMMENT_PROMPT = """
Analyze ONE Persian user comment and return ONE JSON object.

Rules:
- Output ONLY JSON. No markdown. No explanation.
- Use ONLY the comment text.
- evidence MUST be an exact quote from the comment.
- short_title: max 10 words.
- If type != issue → severity = null
- If type != suggestion → priority = null
- ALL fields must be in Persian (fa).
- normalized_title MUST be Persian.
- keywords MUST be Persian.

Allowed values:

type: issue | suggestion | question | praise | other
category: transfer | auth | card | bill | loan | login | ui | performance | AI assistant | other
severity / priority: high | medium | low | null

JSON format:

{{
  "comment_id": "{comment_id}",
  "created_at": "{created_at}",
  "sentiment_group": "{sentiment_group}",

  "type": "",
  "category": "",

  "short_title": "",
  "normalized_title": "",

  "keywords": [],

  "severity": null,
  "priority": null,

  "evidence": "",

  "model": "phi4",
  "processed_at": ""
}}

Comment:
{comment_text}
"""


In [ ]:
import json
import re

ALLOWED_TYPES = {"issue","suggestion","question","praise","other"}
ALLOWED_CATEGORIES = {
    "transfer","auth","card","bill","loan","login","ui","performance", "AI assistant", "other"
}
LEVELS = {"high","medium","low",None}


def validate_output(obj: dict, original_text: str):
    assert obj["type"] in ALLOWED_TYPES
    assert obj["category"] in ALLOWED_CATEGORIES
    assert obj["severity"] in LEVELS
    assert obj["priority"] in LEVELS

    # evidence must be exact substring
    assert obj["evidence"] in original_text

    # no English hallucination
    assert not re.search(r"[A-Za-z]", obj["evidence"])

    for field in ["short_title", "normalized_title"]:
        assert not re.search(r"[A-Za-z]", obj[field]), f"English in {field}"

    for kw in obj["keywords"]:
        assert not re.search(r"[A-Za-z]", kw), "English keyword detected"



    # title length
    assert len(obj["short_title"].split()) <= 12


In [4]:
def append_jsonl(path: str, obj: dict):
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")


In [5]:
import re
import json

def extract_json(raw: str) -> dict:
    if not raw or not raw.strip():
        raise ValueError("Empty LLM output")

    # Remove markdown fences
    raw = re.sub(r"```(?:json)?", "", raw)
    raw = raw.replace("```", "").strip()

    # Extract first JSON object
    match = re.search(r"\{[\s\S]*\}", raw)
    if not match:
        raise ValueError(f"No JSON object found:\n{raw}")

    return json.loads(match.group(0))

In [11]:
from datetime import datetime, timezone
import json

def process_comments_batch(comments, output_path):
    """
    comments: list of dicts with keys:
      id, description, sentiment_group, created_at
    """

    for c in comments:
        try:
            raw = call_phi4_single_comment(
                comment_id=str(c["id"]),
                comment_text=c["description"],
                sentiment_group=c["sentiment_group"],
                created_at=c["created_at"], retries=2
            )

            obj = extract_json(raw)
            obj["processed_at"] = datetime.now(timezone.utc).isoformat()

            validate_output(obj, c["description"])

            append_jsonl(output_path, obj)

        
            if not raw or not raw.strip():
                raise RuntimeError("LLM returned empty output")


        except Exception as e:
            print(f"❌ Failed comment {c['id']}: {e}")







In [14]:
comments = [
    {
        "id": 1,
        "description": "انتقال وجه خطا میده و حساب مبدا نمایش داده نمیشود.",
        "sentiment_group": "negative",
        "created_at": "2025-01-02T10:12:00Z"
    },
    {
        "id": 2,
        "description": "احراز هویت انجام نمیشه و پیامک نمیاد.",
        "sentiment_group": "negative",
        "created_at": "2025-01-02T11:05:00Z"
    }
]


In [15]:
output_path = "processed_comments.jsonl"

process_comments_batch(
    comments=comments,
    output_path=output_path
)


In [9]:
prompt = PHI4_SINGLE_COMMENT_PROMPT.format(
    comment_id="test_1",
    created_at="2025-01-01T00:00:00Z",
    sentiment_group="negative",
    comment_text="انتقال وجه خطا میده و حساب مبدا نمایش داده نمی‌شود."
)

raw = llm.invoke(prompt)
print(raw)


```json
{
  "comment_id": "test_1",
  "created_at": "2025-01-01T00:00:00Z",
  "sentiment_group": "negative",

  "type": "issue",
  "category": "transfer",

  "short_title": "انتقال وجه خطا میده",
  "normalized_title": "خطای انتقال پول",

  "keywords": ["انتقال", "وجه", "خطا", "حساب", "مبدا"],

  "severity": null,
  "priority": null,

  "evidence": "انتقال وجه خطا میده و حساب مبدا نمایش داده نمی‌شود.",
  
  "model": "phi4",
  "processed_at": ""
}
```


In [10]:
with open("processed_comments.jsonl", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i == 3:
            break
        print(line)


{"comment_id": "1", "created_at": "2025-01-02T10:12:00Z", "sentiment_group": "negative", "type": "issue", "category": "transfer", "short_title": "خطای انتقال وجه و نمایش حساب مبدا", "normalized_title": "خطای انتقال وجه و نمایش حساب مبدا", "keywords": ["انتقال", "وجه", "خطا", "حساب", "مبدا", "نمایش"], "severity": "high", "priority": null, "evidence": "انتقال وجه خطا میده و حساب مبدا نمایش داده نمیشود.", "model": "phi4", "processed_at": "2026-01-06T10:09:49.993091+00:00"}

{"comment_id": "2", "created_at": "2025-01-02T11:05:00Z", "sentiment_group": "negative", "type": "issue", "category": "auth", "short_title": "احراز هویت ناموفق است", "normalized_title": "مشکل در احراز هویت و دریافت پیامک", "keywords": ["احراز هویت", "پیامک", "ناموفق"], "severity": "high", "priority": null, "evidence": "احراز هویت انجام نمیشه و پیامک نمیاد.", "model": "phi4", "processed_at": "2026-01-06T10:10:05.060867+00:00"}



## Summarize all results

In [1]:
from langchain_community.llms import Ollama
from datetime import datetime, timezone
import json
import re
from analyze_comments import logger
from preprocessing_main import preprocess


llm_summarize = Ollama(
    model="phi4:latest",
    # base_url="http://localhost:11434",
    base_url = "http://192.168.0.10:11434",
    temperature=0.1
)


#### Preprocessing text (using prerocessing_main)

def normalize_for_match(text: str) -> str:
    if not text:
        return ""

    text = preprocess(
        text,
        remove_halfspace=True,
        replace_multiple_spaces=True,
        replace_enter_with_space=True
    )
    return text.strip()




/home/mahdi/Sentiment_Analysis_Dima/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_586324/4226579205.py:9: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm_summarize = Ollama(


In [11]:
LLM_SUMMARIZE_COMMENT_PROMPT = """

You are an expert in banking bussiness. You should summarize some input shorted comments by consedering their type, category, title and sentiment results and then extract some good summarizations from them.


Rules:
- Output ONLY JSON.
- Clear all previous input before getting the new ones.
- Use the list of {normalized_title} texts in the specific {type}, {category}, {sentiment_result} and {title} as the inputs of summarization.
- See all of input and summarized them as one output.
- The output should be Persian (fa).
- If neccessary, the output could have one or more sentences (Maximum 100 sentences).
- do not generate title. Use the exact input title.
- If neccessary, merge output sentences to give better responses. 
- ALL fields must be in Persian (fa).
- normalized_title MUST be Persian.
- Be sure summarized comment will be filled with sufficient summarization output.   


JSON format:

{{
  
  "type": "{type}",
  "category": "{category}",
  "title" : "{title}"
  "sentiment_result": "{sentiment_result}",
  "summarized_comment": "",

  
}}

"""


In [12]:


def call_LLM_summarize_comment(
    title: str,
    normalized_title: list,
    sentiment_result: str,
    category: str,
    type: str,
    retries: int,
) -> str:
    # preprocessed_comment = normalize_for_match(text = normalized_title)
    prompt = LLM_SUMMARIZE_COMMENT_PROMPT.format(
        title=title,
        category= category,
        sentiment_result=sentiment_result,
        type = type,
        normalized_title= normalized_title
    )
    for i in range(retries + 1):
        raw = llm_summarize.invoke(prompt)

        if raw and raw.strip():
            return raw

        print(f"⚠️ Empty output, retry {i+1}")

    raise RuntimeError("Phi4 returned empty output after retries")




#################################################################################################


In [4]:


def extract_json(raw: str) -> dict:
    if not raw or not raw.strip():
        raise ValueError("Empty LLM output")

    # Remove markdown fences
    raw = re.sub(r"```(?:json)?", "", raw)
    raw = raw.replace("```", "").strip()

    # Extract first JSON object
    # match = re.search(r"\{[\s\S]*\}", raw)
    match = re.search(r"\{[\s\S]*?\}", raw)
    if not match:
        raise ValueError(f"No JSON object found:\n{raw}")

    return json.loads(match.group(0))

In [5]:

# prompt = LLM_SUMMARIZE_COMMENT_PROMPT.format(
#      title="دریافت تسهیلات",
#         category= "loan",
#         sentiment_result="negative",
#         type = "issue",
#         normalized_title= ["وام تعلق نمی‌گیرد","عدم توانایی در دریافت وام 1-2 میلیون","مشکل درخواست وام جدید برای کاربرانی که وام قبل را تسویه کرده‌اند","احراز هویت ناموفق است"]
# )

# raw = llm.invoke(prompt)
# print(raw)


In [6]:
# result = call_LLM_summarize_comment(
#         title="دریافت تسهیلات",
#         category= "loan",
#         sentiment_result="negative",
#         type = "issue",
#         normalized_title= ["وام تعلق نمی‌گیرد","عدم توانایی در دریافت وام 1-2 میلیون","مشکل درخواست وام جدید برای کاربرانی که وام قبل را تسویه کرده‌اند","احراز هویت ناموفق است"],
#         retries= 2)


# extract_json(result)

In [7]:
# extract_json(result)

In [8]:
def append_jsonl(path: str, obj: dict):
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

In [ ]:
from connect_to_database_func import connect_db
from dotenv import load_dotenv
from logging_config import setup_logger  # Import logger setup function

load_dotenv()

# # Initialize logger
logger = setup_logger(name="dima_comments_analysis", log_file="dima_comment_summarization.log")

# Fetch comments that need sentiment analysis for a specific app
def fetch_comments_to_summarize(type,category, title, sentiment_result):
    logger.info("Fetching comments from 'comments' table for LLM analysis.")
    try:
        conn = connect_db()
        cursor = conn.cursor()

        query = """
            SELECT
                type,
                category,
                title,
                sentiment_result,
                normalized_title
            FROM dima_comments_analysis
            WHERE
                type = %s
               AND category = %s
                AND title = %s
                AND sentiment_result = %s        
            ;
            
        """

        cursor.execute(query,(type,category, title,sentiment_result))
        rows = cursor.fetchall()

        # comments = [
            
        #        r[4]
            
        #     for r in rows
        # ]


        comments = [
            {
                "type": r[0],
                "category": r[1],
                "title": r[2],
                "sentiment_result": r[3],
                "normalized_title": r[4]
            }
            for r in rows
        ]
        logger.info(f"Fetched {len(comments)} comments for analysis.")
        count = len(comments)
        cursor.close()
        conn.close()
        return comments , count

    except Exception as e:
        logger.error(f"Error fetching comments: {e}", exc_info=True)
        return []


#################################################################################\
def upsert_summarized_analysis(conn, analysis):
    query = """
        INSERT INTO dima_comments_summarization (
            sentiment_result,
            title,
            type,
            category,
            summarized_comment,
            count,
            processed_at
        )
        VALUES (
            
            %(sentiment_result)s,
            %(title)s,
            %(type)s,
            %(category)s,
            %(summarized_comment)s,
            %(count)s,
            %(processed_at)s
        )
        ON CONFLICT (summarized_id)
        DO UPDATE SET
            sentiment_result = EXCLUDED.sentiment_result,
            title = EXCLUDED.title,
            type = EXCLUDED.type,
            category = EXCLUDED.category,
            summarized_comment = EXCLUDED.summarized_comment,
            count = EXCLUDED.count,
            processed_at = EXCLUDED.processed_at;
    """
    # Ensure the required keys are present in the analysis dictionary
    if 'summarized_comment' not in analysis:
        analysis["summarized_comment"] = "NULL"
        raise KeyError("Missing key: summarized_comment")
    
    with conn.cursor() as cur:
        cur.execute(query, analysis)

# ##########################################################################

# def mark_comment_as_analyzed(conn, comment_id):
#     with conn.cursor() as cur:
#         cur.execute(
#             "UPDATE comments SET is_analyzed = TRUE WHERE id = %s;",
#             (comment_id,)
#         )

In [10]:
result_summarized = fetch_comments_to_summarize(
                title="دریافت تسهیلات",
        category= "card",
        sentiment_result="negative",
        type = "issue"
       )

2026-01-13 09:11:03,821 - INFO - Fetching comments from 'comments' table for LLM analysis.
2026-01-13 09:11:03,821 - INFO - Fetching comments from 'comments' table for LLM analysis.
2026-01-13 09:11:03,821 - INFO - Fetching comments from 'comments' table for LLM analysis.
2026-01-13 09:11:03,841 - INFO - Fetched 6 comments for analysis.
2026-01-13 09:11:03,841 - INFO - Fetched 6 comments for analysis.
2026-01-13 09:11:03,841 - INFO - Fetched 6 comments for analysis.


In [11]:
Final_result = call_LLM_summarize_comment(
        title="دریافت تسهیلات",
        category= "card",
        sentiment_result="negative",
        type = "issue",
        normalized_title= result_summarized,
        retries= 2)

extract_json(Final_result)

{'type': 'issue',
 'category': 'card',
 'title': 'دریافت تسهیلات',
 'sentiment_result': 'negative',
 'normalized_title': 'مشکلات مختلف در انجام عملیات کارت ملی و بروزرسانی موجودی',
 'summarized_comment': 'بر اساس نظرات، مشکلات مختلفی در انجام عملیات کارت ملی به وجود آمده است. برخی از مشکلات شامل تصدیق نشدن کارت ملی، تعریف کارت به عنوان منقضی، خطا در شناسایی سریال کارت در وبسایت، پشتیبانی نادیده گرفتن کارت ملی بدون سریال، خطا در ویرایش شماره سریال کارت ملی و عدم بروزرسانی صحیح موجودی کارت است. این مشکلات باعث نارضایتی و تأخیر در دریافت تسهیلات شده\u200cاند.'}

In [12]:
Final_result

'```json\n{\n  "type": "issue",\n  "category": "card",\n  "title": "دریافت تسهیلات",\n  "sentiment_result": "negative",\n  "normalized_title": "مشکلات مختلف در انجام عملیات کارت ملی و بروزرسانی موجودی",\n  "summarized_comment": "بر اساس نظرات، مشکلات مختلفی در انجام عملیات کارت ملی به وجود آمده است. برخی از مشکلات شامل تصدیق نشدن کارت ملی، تعریف کارت به عنوان منقضی، خطا در شناسایی سریال کارت در وبسایت، پشتیبانی نادیده گرفتن کارت ملی بدون سریال، خطا در ویرایش شماره سریال کارت ملی و عدم بروزرسانی صحیح موجودی کارت است. این مشکلات باعث نارضایتی و تأخیر در دریافت تسهیلات شده\u200cاند."\n}\n```'

In [13]:
result_summarized

[{'type': 'issue',
  'category': 'card',
  'title': 'دریافت تسهیلات',
  'sentiment_result': 'negative',
  'normalized_title': 'مدیریت کارت انجام نمی\u200cشود'},
 {'type': 'issue',
  'category': 'card',
  'title': 'دریافت تسهیلات',
  'sentiment_result': 'negative',
  'normalized_title': 'کارت ملی به عنوان منقضی در نظر گرفته شد'},
 {'type': 'issue',
  'category': 'card',
  'title': 'دریافت تسهیلات',
  'sentiment_result': 'negative',
  'normalized_title': 'خطای شناسایی سریال کارت ملی در وبسایت'},
 {'type': 'issue',
  'category': 'card',
  'title': 'دریافت تسهیلات',
  'sentiment_result': 'negative',
  'normalized_title': 'کارت ملی بدون سریال پشتیبانی نمی\u200cشود'},
 {'type': 'issue',
  'category': 'card',
  'title': 'دریافت تسهیلات',
  'sentiment_result': 'negative',
  'normalized_title': 'خطا در ویرایش شماره سریال کارت ملی'},
 {'type': 'issue',
  'category': 'card',
  'title': 'دریافت تسهیلات',
  'sentiment_result': 'negative',
  'normalized_title': 'موجودی کارت بروزرسانی نمیشه'}]

In [14]:
titles = ["دریافت تسهیلات","انتقال وجه","سایر","مدیریت حساب‌ها","کارت‌ها","دستیار هوشمند","پرداخت قبض","خرید شارژ"]
types = ["issue","suggestion","question","praise","other"]
categories = ["transfer","auth","card","bill","loan","login","ui","performance", "AI assistant", "other"]
sentiments = ["very negative", "negative", "no sentiment expressed","positive", "very positive"]
from datetime import datetime, timezone
import time
# titles = ["دریافت تسهیلات","انتقال وجه"]
# types = ["issue","suggestion"]
# categories = ["transfer","auth","card","loan"]
# sentiments = ["very negative", "negative"]

# titles = ["انتقال وجه"]
# types = ["issue","suggestion"]
# categories = ["transfer","card"]
# sentiments = ["very negative", "negative"]

output =[]
output_path = "summarized_comments.jsonl"
conn = connect_db()
for title_name in titles:
    for type_name in types:
        for category_name in categories:
            for sentiment_name in sentiments:
                result_summarized, count = fetch_comments_to_summarize(
                    title= title_name,
                    category= category_name,
                    sentiment_result= sentiment_name,
                    type = type_name
                    )
                if count <=5:

                    continue

                else: 
                    Final_result = call_LLM_summarize_comment(
                            title= title_name,
                            category= category_name,
                            sentiment_result= sentiment_name,
                            type = type_name,
                            normalized_title= result_summarized,
                            retries= 2)

                    result = extract_json(Final_result)
                    result["title"] = title_name
                    result["type"] = type_name
                    result["category"] = category_name
                    result["sentiment_result"] = sentiment_name
                    result["processed_at"] = datetime.now(timezone.utc).isoformat()
                    result["count"] = count
                    
                    
                    if 'summarized_comment' not in result:
                        logger.warning(f"Missing 'summarized_comment' in the result. Skipping upsert.")
                    else:
    
                        with conn: 
                            upsert_summarized_analysis(conn, result)
                            logger.info(f"summarized text is inserted to summarized analysis table properly")
                        
                    time.sleep(3)  # Sleep for 1 second between each call





                append_jsonl(output_path , result )
                output.append(result)


conn.close()  

2026-01-13 13:10:36,374 - INFO - Fetching comments from 'comments' table for LLM analysis.


2026-01-13 13:10:36,374 - INFO - Fetching comments from 'comments' table for LLM analysis.
2026-01-13 13:10:36,394 - INFO - Fetched 95 comments for analysis.
2026-01-13 13:10:36,394 - INFO - Fetched 95 comments for analysis.
2026-01-13 13:10:49,217 - INFO - summarized text is inserted to summarized analysis table properly
2026-01-13 13:10:49,217 - INFO - summarized text is inserted to summarized analysis table properly
2026-01-13 13:10:52,219 - INFO - Fetching comments from 'comments' table for LLM analysis.
2026-01-13 13:10:52,219 - INFO - Fetching comments from 'comments' table for LLM analysis.
2026-01-13 13:10:52,242 - INFO - Fetched 240 comments for analysis.
2026-01-13 13:10:52,242 - INFO - Fetched 240 comments for analysis.
2026-01-13 13:11:05,381 - INFO - summarized text is inserted to summarized analysis table properly
2026-01-13 13:11:05,381 - INFO - summarized text is inserted to summarized analysis table properly
2026-01-13 13:11:08,383 - INFO - Fetching comments from 'comm

In [15]:
output

[{'type': 'issue',
  'category': 'transfer',
  'title': 'دریافت تسهیلات',
  'sentiment_result': 'very negative',
  'normalized_title': 'مشکلات مختلف در فرایند دریافت و استعلام تسهیلات بانکی',
  'summarized_comment': 'کاربران با موضوعات مختلفی در زمینه دریافت تسهیلات مواجه شده\u200cاند که از جمله آنها مشکلات پرداخت و استعلام چک برگشتی، تأخیر در پرداخت مبالغ، خطای داده در اعتبارسنجی، عدم رسیدگی مناسب پشتیبانی و هزینه\u200cهای بالاتر کارمزد است. چندین نفر به تأخیر طولانی در اپدیت اطلاعات حساب، مشکلات در روند احراز هویت و عدم بروزرسانی سیستم\u200cهای تسویه پرداخت کرده\u200cاند. علاوه بر این، نارضایتی از مبالغ غیر مناسب کسر شده و رد تراکنش\u200cها به دلیل عدم بدهکاری گزارش شده است. در مجموع، این نظرات نشان\u200cدهنده نارضایتی قابل توجه و نیاز به بهبود فرآیندهای مرتبط با تسهیلات بانکی دارد.'},
 {'type': 'issue',
  'category': 'transfer',
  'title': 'دریافت تسهیلات',
  'sentiment_result': 'negative',
  'summarized_comment': 'کاربران با مشکلات مختلف در دریافت و پرداخت تسهیلات روبه\u200cرو شده\